In [4]:
import requests

def inserta_usuarios(datos, api_token):

    usuarios_insertados = 0
    
    for usuario in datos:
        response = requests.post("https://gorest.co.in/public/v2/users",
                                headers={'Authorization': f'Bearer {api_token}'},
                                data=usuario)
    
        if response.status_code == 201:
            usuarios_insertados += 1

    return usuarios_insertados





In [5]:
with open('token_gorest.txt', 'r', encoding='utf8') as f:
    TOKEN_GOREST = f.read().strip()
    # print(TOKEN_GOREST)

u1 = {'name': 'Eva', 'email': 'eva@gmail.com', 'gender': 'female', 'status': 'inactive'}
u2 = {'name': 'Ana', 'email': 'ana@gmail.com', 'gender': 'female', 'status': 'active'}
u3 = {'name': 'Pepe', 'email': 'pepe@gmail.com', 'gender': 'female', 'status': 'inactive'}

api_token = TOKEN_GOREST

print(inserta_usuarios([u1, u2], api_token))

print(inserta_usuarios([u3, u2], api_token))


0
0
